# Hands-on Modul 2.8: Evaluasi Otomatis RAG dengan RAGAS

Evaluasi manual itu lambat dan bias. Di hands-on ini, kita akan menggunakan **RAGAS (Retrieval Augmented Generation Assessment)** untuk mengaudit kualitas jawaban chatbot secara otomatis menggunakan LLM sebagai Juri.

Kita akan fokus pada dua metrik terpenting:
1.  **Faithfulness:** Apakah jawaban model jujur pada dokumen sumber? (Cek Halusinasi)
2.  **Answer Relevancy:** Apakah jawaban model nyambung dengan pertanyaan user?

In [1]:
# Instalasi Ragas dan dependensinya
!pip install ragas openai datasets pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99

In [4]:
import os
from datasets import Dataset

# --- KONFIGURASI API ---
# Masukkan API Key OpenAI Anda di sini JIKA PUNYA.
# Jika tidak punya, biarkan saja "sk-..." atau kosongkan.
# Kode akan otomatis mendeteksi dan masuk ke Mode Simulasi.
os.environ["OPENAI_API_KEY"] = "sk-..."

# --- PERSIAPAN DATA ---
data_samples = {
    'question': [
        'Siapa penemu lampu pijar?',
        'Apa ibukota Indonesia?',
        'Bagaimana cara reset password?'
    ],
    'answer': [
        'Thomas Edison mengembangkan lampu pijar komersial.',
        'Bali adalah ibukota Indonesia yang terkenal wisatanya.', # Halusinasi
        'Mohon maaf, saya tidak tahu.' # Refusal
    ],
    'contexts': [
        ['Thomas Alva Edison mempatenkan lampu pijar pada 1879...'],
        ['Jakarta adalah ibukota negara Indonesia, pusat pemerintahan...'],
        ['Untuk reset password, buka menu setting lalu klik lupa password...']
    ],
    'ground_truth': [
        'Thomas Edison',
        'Jakarta',
        'Buka menu setting -> lupa password'
    ]
}

dataset = Dataset.from_dict(data_samples)
print("Dataset Evaluasi Siap!")

Dataset Evaluasi Siap!


In [5]:
import pandas as pd
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

# Cek apakah user punya Key Valid (Bukan kosong, bukan placeholder)
api_key = os.environ.get("OPENAI_API_KEY", "")
has_valid_key = api_key and api_key != "sk-..." and not api_key.startswith("sk-proj-placeholder")

if has_valid_key:
    print("✅ API Key terdeteksi. Memulai Evaluasi Ragas (Live)...")
    try:
        results = evaluate(
            dataset=dataset,
            metrics=[faithfulness, answer_relevancy],
        )
        df = results.to_pandas()
        print("\n--- Laporan Evaluasi (Live) ---")
        display(df[['question', 'answer', 'faithfulness', 'answer_relevancy']])
    except Exception as e:
        print(f"❌ Terjadi error saat menghubungi OpenAI: {e}")
        print("⚠️ Beralih ke Mode Simulasi...")
        has_valid_key = False # Force fallback

if not has_valid_key:
    print("ℹ️ Mode Simulasi (Mock Data).")
    print("   (Menggunakan data dummy karena tidak ada API Key aktif)")

    # Simulasi Hasil Akhir
    mock_data = data_samples.copy()
    # Skor kita isi manual sesuai skenario belajar
    # Kasus 1: Bagus | Kasus 2: Halusinasi | Kasus 3: Tidak Relevan
    mock_data['faithfulness'] = [0.99, 0.01, 0.00]
    mock_data['answer_relevancy'] = [0.95, 0.80, 0.10]

    df = pd.DataFrame(mock_data)
    print("\n--- Laporan Evaluasi (Simulasi) ---")
    # Kita gunakan print biasa jika display() tidak tersedia di lingkungan lokal
    try:
        display(df[['question', 'answer', 'faithfulness', 'answer_relevancy']])
    except:
        print(df[['question', 'answer', 'faithfulness', 'answer_relevancy']])

ℹ️ Mode Simulasi (Mock Data).
   (Menggunakan data dummy karena tidak ada API Key aktif)

--- Laporan Evaluasi (Simulasi) ---


,question,answer,faithfulness,answer_relevancy
0,Siapa penemu lampu pijar?,Thomas Edison mengembangkan lampu pijar komers...,0.99,0.95
1,Apa ibukota Indonesia?,Bali adalah ibukota Indonesia yang terkenal wi...,0.01,0.80
2,Bagaimana cara reset password?,"Mohon maaf, saya tidak tahu.",0.00,0.10


### Analisis Laporan

Mari kita bedah hasil (Simulasi/Real) di atas:

**Kasus 1: "Penemu Lampu"**
* **Faithfulness: 0.99 (Tinggi)** -> Jawaban sesuai dokumen.
* **Relevancy: 0.95 (Tinggi)** -> Menjawab pertanyaan user.
* **Status:** ✅ PASS.

**Kasus 2: "Ibukota Indonesia"**
* **Faithfulness: 0.01 (Rendah)** -> Model jawab "Bali", Dokumen bilang "Jakarta".
* **Diagnosa:** **Halusinasi**. Model mengabaikan dokumen (*context*) dan mengarang bebas.
* **Status:** ❌ FAIL (Critical Safety Issue).

**Kasus 3: "Reset Password"**
* **Faithfulness: N/A (atau rendah)** -> Model jawab "Tidak tahu", tidak ada di dokumen.
* **Relevancy: 0.1 (Rendah)** -> User tanya cara, model bilang tidak tahu.
* **Diagnosa:** **Refusal / Retrieval Failure**. Model gagal menjawab padahal dokumennya ada.
* **Status:** ⚠️ WARNING (Perlu perbaikan prompt/retriever).**bold text**

### Tantangan Eksperimen (Coba Sendiri!)

Jangan hanya melihat hasil ini. Cobalah modifikasi kode di **Cell 3** atau **Cell 4** untuk melihat bagaimana metrik berubah:

**1. Simulasi "Model Sempurna" (Mode Mock):**
* Pergi ke **Cell 4** (bagian `else / Mode Simulasi`).
* Ubah list skor `mock_data['faithfulness']` menjadi `[1.0, 1.0, 1.0]`.
* Jalankan ulang sel tersebut. Lihat bagaimana laporan berubah menjadi "hijau" semua. Ini adalah target yang ingin kita capai di produksi.

**2. Uji Deteksi Halusinasi (Mode Live API):**
* Jika Anda punya API Key, pergi ke **Cell 3**.
* Ubah `answer` pada pertanyaan pertama menjadi kebohongan: *"Penemu lampu pijar adalah Iron Man."*
* Jalankan ulang **Cell 4**. Perhatikan apakah skor `faithfulness` turun drastis? (Seharusnya turun, karena "Iron Man" tidak ada di `contexts`).

**3. Uji Relevansi (Mode Live API):**
* Ubah `answer` pada pertanyaan "Apa ibukota Indonesia?" menjadi kalimat yang benar tapi tidak nyambung: *"Nasi goreng adalah makanan enak."*
* Jalankan evaluasi. Anda akan melihat `faithfulness` mungkin tinggi (jika tidak bertentangan dengan konteks), tapi `answer_relevancy` akan hancur.

Selamat bereksperimen menjadi "Auditor AI"!